In [1]:
import tensorflow as tf
import numpy as np

2021-11-29 19:23:29.618972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-29 19:23:29.618995: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import pickle

In [3]:
with open('../X_smallset.pkl', 'rb') as inp:
    inputs = pickle.load(inp)

In [4]:
single_input = inputs[0]

In [5]:
test_input = [np.array([[0, 1, 1, 0, 0, 1.5],
             [1, 0, 0, 1, 0, 1.2],
             [1, 0, 0, 1, 0, 1.4]]),
             [[[1, [0, 0, 1]], [2, [0, 1, 0]]],
             [[0, [0, 0, 1]]],
             [[0, [0, 1, 0]]]],
             np.array([3, 2, 4, 7])]

In [6]:
features = single_input[0]
adjacency_list = single_input[1]
physics_features = single_input[2]

In [10]:
w_s = tf.Variable(initial_value=tf.initializers.glorot_uniform()(shape=[81, 80], dtype='double'),shape=[81, 80])
w_n = tf.Variable(initial_value=tf.initializers.glorot_uniform()(shape=[103, 80], dtype='double'),shape=[103, 80])

In [11]:
features[0][-3:]

array([0.031, 1.55 , 0.452])

In [12]:
combination_orders = ([[0, -6], tf.math.add], [[-6, -3], 'position'], [[-3], tf.multiply])

In [13]:
def AtomDistance(x, y):
    return tf.sqrt(tf.reduce_sum(tf.square(x - y)))

In [14]:
new_features = len(features)*[None]
for i, adj in enumerate(adjacency_list):
    self_features = tf.reshape(features[i], [1, 81])
    self_conv_features = tf.matmul(self_features, w_s)
    new_features[i] = self_conv_features
    for neighbour in adj:
        neighbour_features = features[neighbour[0]]
        neighbour_bond = neighbour[1]
        new_ordered_features = []
        distance = None
        for j, order in enumerate(combination_orders):
            order_function = order[1]
            indices = order[0]
            if j == len(combination_orders) - 1 and len(indices) == 1:
                new_ordered_features.append(order_function(self_features[0][indices[0]:], 
                                                                 neighbour_features[indices[0]:]))
            else:
                if order_function == 'position':
                    distance = AtomDistance(self_features[0][indices[0]:indices[1]], 
                                                                   neighbour_features[indices[0]:indices[1]])
                    new_ordered_features.append(neighbour_features[indices[0]:indices[1]])
                else:
                    new_ordered_features.append(order_function(self_features[0][indices[0]:indices[1]], 
                                                                   neighbour_features[indices[0]:indices[1]]))
        new_ordered_features = tf.concat(new_ordered_features, axis=0)
        if distance != None:
            new_ordered_features /= distance**2
        new_ordered_features = tf.concat([new_ordered_features, neighbour_bond], axis=0)
        new_ordered_features = tf.reshape(new_ordered_features, [1, 103])
        new_features[i] += tf.matmul(new_ordered_features, w_n)

In [24]:
import tensorflow as tf

class MultiGraphConvLayer(tf.keras.Model):
    def __init__(self,
                 out_channel,
                 num_features=81,
                 num_bond=22,
                 activation_fn=None,
                 combination_rules = None
                 ):
        super(MultiGraphConvLayer, self).__init__()
        self.out_channel = out_channel
        self.num_features = num_features
        self.num_bond = num_bond
        self.activation_fn = activation_fn
        self.combination_rules = combination_rules
        self.w_s = tf.Variable(initial_value=tf.initializers.glorot_uniform()
        (shape=[num_features, out_channel]), shape=[num_features, out_channel], trainable=True)
        self.w_n = tf.Variable(initial_value=tf.initializers.glorot_uniform()
        (shape=[num_features + num_bond, out_channel]),
                               shape=[num_features + num_bond, out_channel], trainable=True)

    def AtomDistance(x, y):
        return tf.sqrt(tf.reduce_sum(tf.square(x - y)))

    def addRule(self, rule, start_index, end_index = None):
        rules_dict = {
            "sum": tf.math.add,
            "multiply": tf.math.multiply,
            "distance": "distance",
            "divide": tf.math.divide,
            "subtract": tf.math.subtract,
        }
        if type(rule) == str:
            rule = rules_dict[rule]
        if end_index == None:
            self.combination_rules.append([[start_index], rule])
        else:
            self.combination_rules.append([[start_index, end_index], rule])

    def _call_single(self, inp):
        features = inp[0]
        adjacency_list = inp[1]
        new_features = len(features) * [None]
        for i, adj in enumerate(adjacency_list):
            self_features = tf.reshape(features[i], [1, self.num_features])
            self_conv_features = tf.matmul(self_features, self.w_s)
            new_features[i] = self_conv_features
            for neighbour in adj:
                neighbour_features = features[neighbour[0]]
                neighbour_bond = neighbour[1]
                new_ordered_features = []
                distance = None
                for j, rule in enumerate(self.combination_rules):
                    rule_function = rule[1]
                    indices = rule[0]
                    if j == len(self.combination_rules) - 1 and len(indices) == 1:
                        new_ordered_features.append(rule_function(self_features[0][indices[0]:],
                                                                  neighbour_features[indices[0]:]))
                    else:
                        if rule_function == 'distance':
                            distance = self.AtomDistance(self_features[0][indices[0]:indices[1]],
                                                         neighbour_features[indices[0]:indices[1]])
                            new_ordered_features.append(neighbour_features[indices[0]:indices[1]])
                        else:
                            new_ordered_features.append(rule_function(self_features[0][indices[0]:indices[1]],
                                                                      neighbour_features[indices[0]:indices[1]]))
                new_ordered_features = tf.concat(new_ordered_features, axis=0)
                if distance != None:
                    new_ordered_features /= distance ** 2
                new_ordered_features = tf.concat([new_ordered_features, neighbour_bond], axis=0)
                new_ordered_features = tf.reshape(new_ordered_features, [1, self.num_features + self.num_bond])
                new_features[i] += tf.matmul(new_ordered_features, self.w_n)
        return ([new_features, adjacency_list, inp[2:]])

    def call(self, inputs):
        return tf.map_fn(fn = lambda i: self._call_single(i), elems = inputs)



In [21]:
combination_orders = ([[0, -6], tf.math.add], [[-6, -3], 'distance'], [[-3], tf.multiply])
m = MultiGraphConvLayer(80)
m.addRule("sum", 0, -6)
m.addRule("distance", -6, -3)
m.addRule("multiply", -3)

In [22]:
m(inputs)

ValueError: Exception encountered when calling layer "multi_graph_conv_layer" (type MultiGraphConvLayer).

Shape () must have rank at least 1

Call arguments received:
  • inputs=[['tf.Tensor(shape=(1724, 81), dtype=float32)', [[['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)']], [['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)']], [['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)']], [['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)']], [['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shap

In [85]:
tf.matmul(a, b)

InvalidArgumentError: cannot compute MatMul as input #1(zero-based) was expected to be a int32 tensor but is a float tensor [Op:MatMul]